<a href="https://colab.research.google.com/github/vicente-gonzalez-ruiz/PRMC-experimentos/blob/master/src/shift_invariance_in_OLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Can the LP represented critically?
Not.

In [ ]:
%matplotlib inline
from io_stuff import *
#import cv2
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
!ln -sf ~/MRVC/src/LP.py .
!ln -sf ~/MRVC/src/config.py .
!ln -sf ~/MRVC/src/YCoCg.py .
!ln -sf ~/MRVC/src/YCrCb.py .
import LP
import YCoCg as YUV
#import YCrCb as color_transform

In [ ]:
%%bash

# Create the two images ("/tmp/moving_circle_000.png" and "/tmp/moving_circle_001.png") with a circle
# of diameter 10 that moves from coordinate (20,30) to (21,30).
/home/vruiz/MRVC/tools/moving_circle.sh -w 128 -h 128 -x 20 -y 30 -d 10 -f 2

In [ ]:
RGB_moving_circle_000 = cv2.imread("/tmp/moving_circle_000.png").astype(np.int16)
RGB_moving_circle_001 = cv2.imread("/tmp/moving_circle_001.png").astype(np.int16)

In [ ]:
plt.figure(figsize=(10,10))
plt.title("/tmp/moving_circle_000.png", fontsize=20)
plt.imshow(RGB_moving_circle_000)

In [ ]:
plt.figure(figsize=(10,10))
plt.title("/tmp/moving_circle_001.png", fontsize=20)
plt.imshow(RGB_moving_circle_001)

In [ ]:
YUV_moving_circle_000 = YUV.from_RGB(RGB_moving_circle_000)
YUV_moving_circle_001 = YUV.from_RGB(RGB_moving_circle_001)

In [ ]:
plt.figure(figsize=(10,10))
plt.title("/tmp/moving_circle_000.png", fontsize=20)
plt.imshow(YUV_moving_circle_000[:,:,0], cmap="gray")

In [ ]:
plt.figure(figsize=(10,10))
plt.title("/tmp/moving_circle_001.png", fontsize=20)
plt.imshow(YUV_moving_circle_001[:,:,0], cmap="gray")

In [ ]:
plt.plot(YUV_moving_circle_000[35, 0:40, 0])
plt.show()
plt.plot(YUV_moving_circle_001[35, 0:40, 0])
plt.show()

In [ ]:
n_levels = 2
lp_000 = LP.analyze(YUV_moving_circle_000, n_levels)
lp_001 = LP.analyze(YUV_moving_circle_001, n_levels)

In [ ]:
def normalize(img):
    max_component = np.max(img)
    min_component = np.min(img)
    max_min_component = max_component - min_component
    return (img-min_component)/max_min_component

def show_frame(frame, title=''):
    frame = normalize(frame)
    plt.figure(figsize=(10,10))
    plt.title(title, fontsize=20)
    plt.imshow(frame)

In [ ]:
for l in range(n_levels+1):
    plt.figure(figsize=(10,10))
    plt.title(f"{l}")
    plt.imshow(normalize(lp_000[l]))
    plt.show()
    plt.figure(figsize=(10,10))
    plt.imshow(normalize(lp_001[l]))
    plt.show()

In [ ]:
for l in range(n_levels+1):
    plt.figure(figsize=(10,10))
    plt.title(f"{l}")
    plt.imshow(lp_000[l][:,:,0], cmap="gray")
    plt.show()
    plt.figure(figsize=(10,10))
    plt.imshow(lp_001[l][:,:,0], cmap="gray")
    plt.show()

### DWT of the base of the pyramid

In [ ]:
import pywt

In [ ]:
LL, (LH, HL, HH) = pywt.dwt2(lp_000[0][:,:,0], "haar")

In [ ]:
plt.figure(figsize=(10,10))
plt.title("/tmp/level 1 (image 000)", fontsize=20)
plt.imshow(LL, cmap="gray")

## Ignore the rest

In [ ]:
%%bash

# The images are in the RGB domain and we work in the YUV domain, so we convert them.
/home/vruiz/MRVC/tools/RGB2YUV.py -i /tmp/moving_circle_000.png -o /tmp/YUV_moving_circle_000.png
/home/vruiz/MRVC/tools/RGB2YUV.py -i /tmp/moving_circle_001.png -o /tmp/YUV_moving_circle_001.png

In [ ]:
# Plot the first part (from pixel 0 to 39) of the horizontal line (row 35) of the luminance.
YUV_moving_circle_000 = cv2.imread("/tmp/YUV_moving_circle_000.png")
YUV_moving_circle_001 = cv2.imread("/tmp/YUV_moving_circle_001.png")
plt.plot(YUV_moving_circle_000[35, 0:40, 0])
plt.show()
plt.plot(YUV_moving_circle_001[35, 0:40, 0])
plt.show()

In [ ]:
# Generate Gaussian pyramid (of 2 levels) for both images
# (https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_pyramids/py_pyramids.html)

levels = 2

G_000 = YUV_moving_circle_000[:,:,0]#.copy()
G_001 = YUV_moving_circle_001[:,:,0]#.copy()
gp_000 = [G_000]
gp_001 = [G_001]
for i in range(levels):
    G_000 = cv2.pyrDown(G_000)
    G_001 = cv2.pyrDown(G_001)
    gp_000.append(G_000)
    gp_001.append(G_001)

In [ ]:
plt.figure(figsize=(10,10))
plt.title("/tmp/level 1 (image 000)", fontsize=20)
plt.imshow(gp_000[0], cmap="gray")

In [ ]:
# Generate Laplacian Pyramid for both images.
lp_000 = [gp_000[levels-1]]
lp_001 = [gp_001[levels-1]]
for i in range(levels-1, 0, -1):
    GE_000 = cv2.pyrUp(gp_000[i])
    GE_001 = cv2.pyrUp(gp_001[i])
    #L_000 = cv2.subtract(gp_000[i-1], GE_000)
    L_000 = gp_000[i-1]- GE_000
    #L_001 = cv2.subtract(gp_001[i-1], GE_001)
    L_001 = gp_001[i-1]- GE_001
    lp_000.append(L_000)
    lp_001.append(L_001)

In [ ]:
plt.figure(figsize=(10,10))
plt.title("/tmp/level 1 (image 000)", fontsize=20)
plt.imshow(lp_000[1], cmap="gray")

In [ ]:
plt.figure(figsize=(10,10))
plt.title("/tmp/level 1 (image 001)", fontsize=20)
plt.imshow(lp_001[1], cmap="gray")